In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [45]:
n_vocab = 6785
n_embed = 300
num_filters = 250
num_classes = 2

In [46]:
class Model(nn.Module):
    '''Deep Pyramid Convolutional Neural Networks for Text Categorization'''
    def __init__(self, n_vocab, n_embed, num_filters, num_classes):
        super(Model, self).__init__()
        
        self.embedding = nn.Embedding(n_vocab, n_embed, padding_idx=n_vocab - 1)
        self.conv_region = nn.Conv2d(1, num_filters, (3, n_embed), stride=1)
        self.conv = nn.Conv2d(num_filters, num_filters, (3, 1), stride=1)
        self.max_pool = nn.MaxPool2d(kernel_size=(3, 1), stride=2)
        self.padding1 = nn.ZeroPad2d((0, 0, 1, 1))  # top bottom
        self.padding2 = nn.ZeroPad2d((0, 0, 0, 1))  # bottom
        self.relu = nn.ReLU()
        self.fc = nn.Linear(num_filters, num_classes)

    def forward(self, x):
        x = x[0]
        x = self.embedding(x)
        x = x.unsqueeze(1)       # [batch_size, 250, seq_len, 1]
        x = self.conv_region(x)  # [batch_size, 250, seq_len-3+1, 1]

        x = self.padding1(x)  # [batch_size, 250, seq_len, 1]
        x = self.relu(x)
        x = self.conv(x)      # [batch_size, 250, seq_len-3+1, 1]
        x = self.padding1(x)  # [batch_size, 250, seq_len, 1]
        x = self.relu(x)
        x = self.conv(x)      # [batch_size, 250, seq_len-3+1, 1]
        while x.size()[2] > 2:
            x = self._block(x)
        print("1----", x.size())
        x = x.squeeze()       # [batch_size, num_filters(250)]
        print("2----", x.size())
        x = self.fc(x)
        print("3----", x.size())
        return x

    def _block(self, x):
        x = self.padding2(x)
        px = self.max_pool(x)

        x = self.padding1(px)
        x = F.relu(x)
        x = self.conv(x)

        x = self.padding1(x)
        x = F.relu(x)
        x = self.conv(x)

        x = x + px
        return x

In [47]:
model = Model(n_vocab, n_embed, num_filters, num_classes)

In [48]:
model

Model(
  (embedding): Embedding(6785, 300, padding_idx=6784)
  (conv_region): Conv2d(1, 250, kernel_size=(3, 300), stride=(1, 1))
  (conv): Conv2d(250, 250, kernel_size=(3, 1), stride=(1, 1))
  (max_pool): MaxPool2d(kernel_size=(3, 1), stride=2, padding=0, dilation=1, ceil_mode=False)
  (padding1): ZeroPad2d(padding=(0, 0, 1, 1), value=0.0)
  (padding2): ZeroPad2d(padding=(0, 0, 0, 1), value=0.0)
  (relu): ReLU()
  (fc): Linear(in_features=250, out_features=2, bias=True)
)

In [63]:
#(x, seq_len), y
# x = torch.LongTensor([_[0] for _ in datas]).to(self.device)
# y = torch.LongTensor([_[1] for _ in datas]).to(self.device)

# # pad前的长度(超过pad_size的设为pad_size)
# seq_len = torch.LongTensor([_[2] for _ in datas]).to(self.device)
# return (x, seq_len), y
pad_size = 2000

In [64]:
data = []
for i in range(128):
    x = torch.randint(0, n_vocab, (pad_size,))
    y = 1
    seq_len = pad_size
    data.append((x, y, seq_len))

In [65]:
x = torch.LongTensor([_[0].numpy() for _ in data])
y = torch.LongTensor([_[1] for _ in data])
seq_len = torch.LongTensor([_[2] for _ in data])
batch = (x, seq_len), y

In [66]:
new_data = (x, seq_len), y

In [67]:
new_data

((tensor([[2641, 3142, 6038,  ..., 4882, 6404, 5904],
          [ 674,  180, 3888,  ..., 5296, 6608, 4757],
          [3255, 2628, 3481,  ...,  403, 3442, 3272],
          ...,
          [5632, 4497, 1870,  ..., 3487, 3319, 1152],
          [6124, 3428, 3456,  ..., 1180,  198, 1373],
          [1043, 2513, 2178,  ..., 6002,  426, 6406]]),
  tensor([2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
          2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
          2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
          2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
          2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
          2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
          2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
          2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000,
   

In [68]:
model(new_data[0])
# 1---- torch.Size([128, 250, 2, 1])
# 2---- torch.Size([128, 250, 2])
## 正常
# 1---- torch.Size([128, 250, 1, 1])
# 2---- torch.Size([128, 250])
# 3---- torch.Size([128, 2])

1---- torch.Size([128, 250, 1, 1])
2---- torch.Size([128, 250])
3---- torch.Size([128, 2])


tensor([[-1.1484,  0.5910],
        [-1.1393,  0.6055],
        [-1.1429,  0.6078],
        [-1.1623,  0.6407],
        [-1.1470,  0.5874],
        [-1.1105,  0.6037],
        [-1.1328,  0.6101],
        [-1.1027,  0.6200],
        [-1.1198,  0.6108],
        [-1.1542,  0.6378],
        [-1.1362,  0.5857],
        [-1.1055,  0.5995],
        [-1.1283,  0.5863],
        [-1.1505,  0.6202],
        [-1.1410,  0.5996],
        [-1.1273,  0.5883],
        [-1.0928,  0.6062],
        [-1.1119,  0.6081],
        [-1.1025,  0.6386],
        [-1.1303,  0.6089],
        [-1.1290,  0.6132],
        [-1.1077,  0.6144],
        [-1.1063,  0.5967],
        [-1.0977,  0.6082],
        [-1.1522,  0.5888],
        [-1.1152,  0.6270],
        [-1.1378,  0.5977],
        [-1.1371,  0.5990],
        [-1.1483,  0.6291],
        [-1.1017,  0.6201],
        [-1.1417,  0.5767],
        [-1.1428,  0.5843],
        [-1.1030,  0.5875],
        [-1.1252,  0.6312],
        [-1.1298,  0.6594],
        [-1.1188,  0

In [33]:
new_data[0][0].size()

torch.Size([128, 1500])